In [1]:
import os
import glob
import numpy as np
from nd2reader import ND2Reader
import napari
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot
import time
import pandas as pd
import tifffile as tiff
from skimage import io
import plotly.express as px

In [2]:
dir1 = "./" ##Input Directory with the ND2 files
dir3 = dir1+'TiffFiles'

if not os.path.isdir(dir3):
    os.mkdir(dir3)

files=[]
files += [each for each in os.listdir(dir1) if each.endswith('.nd2')]
files = np.array(files)
files.sort()
string = 'Bleach'
mask = np.array([string not in s for s in files])

files = files[mask]



### Saving Files as OME Tiffs

- All images are converte to .tif files and saved in the TiffFiles folder
- Unclear yet how to change the LUTs

In [ ]:
from nis2pyr.convertor import convert_nd2_to_pyramidal_ome_tiff

for fil in files:
    print(dir1+fil)
    convert_nd2_to_pyramidal_ome_tiff(dir1+fil, dir3+'/'+fil.split(".")[0]+'.tif',max_levels=1)


### Maximum Intensity Projection and Image Distribution according to FOV 
So all images corresponding to the same FOV but across different cycles are saved in one folder 
- each image for one round is split by its channels and one channel image is saved
- still need to figure out how to manage when there are multiple channels

In [ ]:
dir4 = dir3+'/Max_Projections/'
dir50 = dir4+'FOVs/'
if not os.path.isdir(dir50):
    os.mkdir(dir50)

files=[]
files += [each for each in os.listdir(dir3) if each.endswith('.tif')]
files = np.array(files)
files.sort()

if not os.path.isdir(dir4):
    os.mkdir(dir4)

def maximum_intensity_projection(image_stack):
    # Calculate the maximum intensity projection along the Z-axis (channels are in the first dimension)
    mip = np.max(image_stack, axis=0)
    return mip

for fil in files:
    #print(fil)
    ome_tiff_path = dir3+"/"+fil
    image_stack = tiff.imread(ome_tiff_path)
    mip = maximum_intensity_projection(image_stack)
    mip_path = dir4+"MAX_"+fil.split(".")[0]+".tif"
    tiff.imsave(mip_path, mip)
    FOV = fil.split("_")
    dir5 = dir50+"/"+FOV[1].split(".")[0]+"/"
    if not os.path.isdir(dir5):
        os.mkdir(dir5)
    for chan in range(0,mip.shape[0]):
        image = mip[chan,:,:]
        savename =  dir5+FOV[0]+"_channel_"+str(chan+1)+".tif"
        tiff.imsave(savename, image)
    

### Creating Files combining images from single channel and single FOV across all cycles
- This helps in easy visualization of the data and can also be used for data analysis? 
- the files are saved in the Max_Projection/Combined_Files/ folder

In [ ]:
directories = [name for name in os.listdir(dir4) if os.path.isdir(os.path.join(dir4, name))]
directories.sort()

savedir = dir4+"/"+"Combined_Files/"
if not os.path.isdir(savedir):
    os.mkdir(savedir)

for dir in directories:
    fils=[]
    dirs = dir4+"/"+dir
    fils += [each for each in os.listdir(dirs) if each.endswith('.tif')]
    fils = np.array(fils)
    fils.sort()
    chan = np.char.split(fils,sep="_")
    chan = pd.DataFrame(np.stack(chan,axis=0))[2]
    chans = pd.unique(chan)
    for c in chans:
        index = chan==c
        multifils = fils[index]
        multichannel_image = np.zeros((len(multifils),image.shape[0], image.shape[1]))
        round=0
        for m in multifils:
            img = tiff.imread(dirs+"/"+m)
            multichannel_image[round,:,:] = img
            round=round+1
        savename = savedir+dir+"_channel_"+c
        tiff.imsave(savename, multichannel_image)
            

### FISH Spot Detection

##### Detecting threshold within multiple FOVs for all rounds and channels
- This could potentially be used to get a global threshold value
- Alternatively can be used to get individual thresholds for each channel and round

In [ ]:
dir4 = dir3+'/Max_Projections/'
dir50 = dir4+'FOVs/'

start_time = time.time()

voxelval = 110.3752759382
radiusval = 250.0

thresh1=18 ## Threshold for Channel 1 (generally Cy7)
thresh2=18 ## Threshold for Channel 2 (generally Cy5)
thresh3=18 ## Threshold for Channel 3 (generally Cy3B)

#Number of Cycles for each channel
chan1round = 8
chan2round = 8
chan3round = 8

## Number of FOVs to use to detect thresholds
nfovs = 15

dir10 = dir4+"/"+"Combined_Files/"
files=[]
files += [each for each in os.listdir(dir10) if each.endswith('.tif')]
files = np.array(files)
files.sort()
chan = np.char.split(files,sep="_")
chan = pd.DataFrame(np.stack(chan,axis=0))[2]
chans = pd.unique(chan)

ths1 = np.zeros((chan1round*nfovs, 1))
ths2 = np.zeros((chan2round*nfovs+1, 1))
ths3 = np.zeros((chan3round*nfovs, 1))

spot_radius_px = detection.get_object_radius_pixel(
                    voxel_size_nm=(voxelval, voxelval), 
                    object_radius_nm=(radiusval, radiusval), 
                    ndim=2)


e = 0

## Channel 1 - Cy7
for fil in range(0,nfovs):

    print("--- Start %s seconds ---" % (time.time() - start_time))
    filename = files[fil*len(chans)]
    img = tiff.imread(dir10+filename)
    print(filename)
    ### Detect spots
    for t in range(0,img.shape[0]):
        rna = img[t,:,:]    
        # LoG filter
        rna_log = stack.log_filter(rna, sigma=spot_radius_px)
        # local maximum detection
        mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)
        # thresholding
        threshold = detection.automated_threshold_setting(rna_log, mask)
        ths1[e] = threshold
        e=e+1
       
print("Finished thresholding for channel 1 after %s seconds ---" % (time.time() - start_time))       

## Channel 2 - Cy5
        
e = 0
for fil in range(0,nfovs):

    print("--- Start %s seconds ---" % (time.time() - start_time))
    filename = files[fil*len(chans)+1]
    img = tiff.imread(dir10+filename)
    print(filename)
    ### Detect spots 
    for t in range(0,img.shape[0]):
        rna = img[t,:,:]    
        # LoG filter
        rna_log = stack.log_filter(rna, sigma=spot_radius_px)
        # local maximum detection
        mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)
        # thresholding
        threshold = detection.automated_threshold_setting(rna_log, mask)
        ths2[e] = threshold
        e=e+1
        
print("Finished thresholding for channel 2 after %s seconds ---" % (time.time() - start_time))               

## Channel 3 - Cy3B

e = 0
for fil in range(0,nfovs):

    print("--- Start %s seconds ---" % (time.time() - start_time))
    filename = files[fil*len(chans)+2]
    img = tiff.imread(dir10+filename)
    print(filename)
    ### Detect spots
    for t in range(0,img.shape[0]):
        rna = img[t,:,:]    
        # LoG filter
        rna_log = stack.log_filter(rna, sigma=spot_radius_px)
        # local maximum detection
        mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)
        # thresholding
        threshold = detection.automated_threshold_setting(rna_log, mask)
        ths3[e] = threshold
        e=e+1

print("Finished thresholding for channel 3 after %s seconds ---" % (time.time() - start_time))       

##### Applying threshold to detect spots and clusters
- spots are saved in the Spots folder as FOV##\_Channel##\_#.csv
- spot clusters are save in the Spots folder as FOV##\_Channel##\_spotclusters_#.csv
- clusters information is save in the Spots folder as FOV##\_Channel##\_clusters\_#.csv


In [ ]:
start_time = time.time()

voxelval = 110.3752759382 ##XY pixel size
radiusval = 2*voxelval  ## size of spot approx - possible we could alter this based on channel??
thresh1=2*np.median(ths1)+40 ## Threshold for Channel 1
thresh2=2*np.median(ths2)+40 ## Threshold for Channel 2
thresh3=2*np.median(ths3)+40 ## Threshold for Channel 3

threshs = [thresh1,thresh2,thresh3]

chan1round = 7
chan2round = 7
chan3round = 6

dir10 = dir4+"/"+"Combined_Files/"
files=[]
files += [each for each in os.listdir(dir10) if each.endswith('.tif')]
files = np.array(files)
files.sort()
chan = np.char.split(files,sep="_")
chan = pd.DataFrame(np.stack(chan,axis=0))[2]
chans = pd.unique(chan)

spot_radius_px = detection.get_object_radius_pixel(
                    voxel_size_nm=(voxelval, voxelval), 
                    object_radius_nm=(radiusval, radiusval), 
                    ndim=2)

nfovs = int(len(files)/len(chans))

for fil in range(0,nfovs):

    print("--- Start %s seconds ---" % (time.time() - start_time))
    for cc in range(0,len(chans)-1):
        print("Analysing Channel %s"%(cc))
        filename = files[fil*len(chans)+cc]
        img = tiff.imread(dir10+filename)
        savenamespots = filename.split(".")[0]+".csv"
        savenamespotscl = filename.split(".")[0]+"_spotclusters.csv"
        savenameclusters = filename.split(".")[0]+"_clusters.csv"
        
        sp = pd.DataFrame()
        spcl = pd.DataFrame()
        cl = pd.DataFrame()
        
        print(filename)
        ### Detect spots
        for t in range(0,img.shape[0]-1):
            print("Analysing Round %s"%(t+1))
            rna = img[t+1,:,:]
            # viewer.add_image(rna,rgb=False,
            #                     name='FOV_'+str(fil)+"_channel_"+str(t+1))
            # LoG filter
            spots = detection.detect_spots(
                                    images=rna, 
                                    return_threshold=False,
                                    threshold=threshs[cc], 
                                    voxel_size=(voxelval, voxelval),  # in nanometer (one value per dimension zyx)
                                    spot_radius=(radiusval, radiusval))  # in nanometer (one value per dimension zyx)

            spots_post_decomposition, dense_regions, reference_spot = detection.decompose_dense(
                                    image=np.uint16(rna), 
                                    spots=spots, 
                                    voxel_size=(voxelval, voxelval), 
                                    spot_radius=(radiusval, radiusval), 
                                    alpha=0.75,  # alpha impacts the number of spots per candidate region
                                    beta=0.9,  # beta impacts the number of candidate regions to decompose
                                    gamma=15)  # gamma the filtering step to denoise the image
            
            spots_post_clustering, clusters = detection.detect_clusters(
                                    spots=spots_post_decomposition, 
                                    voxel_size=(int(voxelval), int(voxelval)), 
                                    radius=int(radiusval), 
                                    nb_min_spots=4)

            spotspd = pd.DataFrame(spots)
            spclpd = pd.DataFrame(spots_post_clustering)
            clupd = pd.DataFrame(clusters)
            
            spotspd['round'] = t
            spclpd['round'] = t
            clupd['round'] = t
                        
            sp = pd.concat([sp,spotspd])
            spcl = pd.concat([spcl,spclpd])
            cl = pd.concat([cl,clupd])
            
        sp.columns = ['Y','X','round']
        cl.columns = ['Y','X','nspots','index','round']
        spcl.columns = ['Y','X','clusterindex','round']

        sp.to_csv(savedir+savenamespots, index=False)
        spcl.to_csv(savedir+savenamespotscl, index=False) 
        cl.to_csv(savedir+savenameclusters, index=False)     
        
    print("Finished thresholding for channel 1 image %s after %s seconds ---" % (e,time.time() - start_time))  
       
   

#### Napari viewer
- View all images in one 

In [ ]:
def nearest_square(limit):
    answer = 0
    while (answer+1)**2 < limit:
        answer += 1
    if answer**2 == limit:
        return answer
    else:
        return answer+1

viewer= napari.Viewer()
#nfovs=2

gap = 2800

totalarea = nearest_square(len(files)/4)
image=np.zeros((gap*totalarea,gap*totalarea),dtype=np.int16)

xx=0
yy=0

dir4 = dir3+'/Max_Projections/'
dir50 = dir4+'FOVs/'
dir10 = dir4+"/"+"Combined_Files/"
dir11 = dir4+"/"+"Spots/"
files=[]
files += [each for each in os.listdir(dir10) if each.endswith('.tif')]
files = np.array(files)
files.sort()
chan = np.char.split(files,sep="_")
chan = pd.DataFrame(np.stack(chan,axis=0))[2]
chans = pd.unique(chan)

genescy7 = ['Dpp10','Dpp8','Dst','Fat1','Itgb4','Larp1','Larp7']
genescy5 = ['Apob','Cdh1','Ctnnb1','Cyb5r3','mTor','Net1','Pigr']
genescy3 = ['Dync1h1','Dync1i2','Eif4h','Kif1c','Kif3b','Kif5b','None']

spotscy7 = pd.DataFrame()
spotscy5 = pd.DataFrame()
spotscy3 = pd.DataFrame()
color = px.colors.qualitative.Light24

for fil in range(int(len(files)/4)):

#for fil in range(0,2):

    imagename = files[len(chans)*fil+3]
    print(imagename)
    imageloc = dir10+imagename
    im2 = tiff.imread(imageloc)
    image[xx*gap:xx*gap+im2.shape[1],yy*gap:yy*gap+im2.shape[1]] = im2[0,:,:]

    cy7spots = pd.read_csv(dir11+files[len(chans)*fil].split(".")[0]+"_spotclustters.csv")
    cy5spots = pd.read_csv(dir11+files[len(chans)*fil+1].split(".")[0]+"_spotclustters.csv")
    cy3spots = pd.read_csv(dir11+files[len(chans)*fil+2].split(".")[0]+"_spotclustters.csv")
    
    cy7spots['Y'] = cy7spots['Y']+xx*gap
    cy7spots['X'] = cy7spots['X']+yy*gap
    cy5spots['Y'] = cy5spots['Y']+xx*gap
    cy5spots['X'] = cy5spots['X']+yy*gap
    cy3spots['Y'] = cy3spots['Y']+xx*gap
    cy3spots['X'] = cy3spots['X']+yy*gap
    
    gency7ind = np.array(cy7spots['round'].values)
    genecy7 = [genescy7[j] for j in gency7ind]

    gency5ind = np.array(cy5spots['round'].values)
    genecy5 = [genescy5[j] for j in gency5ind]
    
    gency3ind = np.array(cy3spots['round'].values)
    genecy3 = [genescy3[j] for j in gency3ind]
    
    cy7spots['gene'] = genecy7
    cy5spots['gene'] = genecy5
    cy3spots['gene'] = genecy3
    
    spotscy7 = pd.concat([spotscy7,cy7spots])
    spotscy5 = pd.concat([spotscy5,cy5spots])
    spotscy3 = pd.concat([spotscy3,cy3spots])
    
    xx = xx+1
    if xx>totalarea-1:
        xx=0
        yy=yy+1
    
imagelayer = viewer.add_image(np.uint16(image))
imagelayer.contrast_limits=(0,65000)

allspots = pd.concat([spotscy7,spotscy5],axis=0)
allspots = pd.concat([allspots,spotscy3],axis=0)

for round in range(0,len(genescy7)):

    cy7 = spotscy7[spotscy7['round']==round]
    cy5 = spotscy5[spotscy5['round']==round]
    cy3 = spotscy3[spotscy3['round']==round]
                
    viewer.add_points(np.array(cy7)[:,0:2],
                        face_color=color[(len(chans)-1)*round],
                        size=5,
                        blending='translucent_no_depth',
                        edge_width=0,
                        name=genescy7[round])

    viewer.add_points(np.array(cy5)[:,0:2],
                        face_color=color[(len(chans)-1)*round+1],
                        size=5,
                        blending='translucent_no_depth',
                        edge_width=0,
                        name=genescy5[round])

    viewer.add_points(np.array(cy3)[:,0:2],
                        face_color=color[(len(chans)-1)*round+2],
                        size=5,
                        blending='translucent_no_depth',
                        edge_width=0,
                        name=genescy3[round])



featuresall = allspots['gene']
feat = allspots['gene'].values

featuresall.to_csv(dir11+'features.csv',index=False)
allspots.to_csv(dir11+"allspots.csv",index=False)

feat = tuple(np.array(featuresall))
    
pointlayer = viewer.add_points(np.array(allspots)[:,0:2],
                                face_color='white',
                                size=5,
                                blending='translucent_no_depth',
                                edge_width=0,
                                name='All_spots',
                                opacity=0,
                                features=feat)    

                        
pointlayer.refresh()